<br><font color = darkblue size=6><strong>K Means Clustering on Thematic Investing</strong></font>

Identify the clusters of mega trends by analyzing thematic ETFs 

Thematic investing is a form of investment which aims to capture mega trends that can grow faster than the average economy over the long run. For example, if we want to capture the mega trend of aging population, then our thematic investing can be buying stocks related to senior housing, health care, etc.    

Although the popularity of thematic investing is <a href="https://www.etftrends.com/thematic-investing-channel/newfound-popularity-of-thematic-etfs/">increasing</a></div>, the categorization of stocks by mega trend is vague. For instance, some ETF providers have offerings on broad mega trends such as Energy Transition, whereas some providers further split this trend into two separate offerings on Smart Mobility and Clean Energy mega trends. 

With so many different definitions on mega trends, understanding the thematic investing landscape can be challenging. It would be nice if we could group these mega trends into a small number of classifications, according to some similarities.

This article illustrates my application of K Means Clustering on grouping the mega trends. The Python implementation of this project is available on my GitHub.

# Data collection

To collect varied definitions on mega trends, I used <a href="https://www.etf.com/channels/theme-investing">etf.com</a></div> database to compile a list of thematic ETFs. At the time of writing, there were 143 ETFs in total. I saved the list in an Excel file named `Thematic ETFs.xlsx` and here shows the first five:

In [1]:
# import libraries
import pandas as pd

# load the list of thematic ETFs
ETF = pd.read_excel('Thematic ETFs.xlsx', skiprows=1, index_col=0)
ETF.head()

,Fund Name,Issuer,AUM,Expense Ratio,3-Mo TR,Segment
Ticker,,,,,,
ARKK,ARK Innovation ETF,ARK,$23.69B,0.0075,-0.1588,Equity: Global Theme Broad-based
ARKW,ARK Next Generation Internet ETF,ARK,$6.79B,0.0079,-0.0806,Equity: Global Internet
ICLN,iShares Global Clean Energy ETF,Blackrock,$5.59B,0.0046,-0.2811,Equity:Global Renewable Energy
GUNR,FlexShares Morningstar Global Upstream Natural...,Northern Trust,$5.05B,0.0046,0.0889,Equity: Global Natural Resources
ARKF,ARK Fintech Innovation ETF,ARK,$4.20B,0.0075,-0.0303,Equity: Global Technology


These ETFs are provided by a wide variety of issuers, 41 in total. This is ideal because we get to capture a wide variety of mega trend definitions.

In [2]:
# check number of issuers
issuers = ETF.groupby('Issuer').size()

# plot the issuers
import cufflinks as cf
issuers.iplot(kind='bar', title='ETF Issuers', yTitle='Number of ETFs')

## Price data

Features related to an ETF could be its price data as well as holdings data. I first downloaded the price data for individual ETFs from Yahoo! Finance. The length of the return history varies across the ETFs. For example, ACES has a history of 713 days, whereas AGNG has 1,252 days. The majority of the ETFs have less than 2 years of history.  

In [3]:
# download price data from Yahoo! Finance
import yfinance as yf
ticker = list(ETF.index)
price = yf.download(ticker, progress=False)['Adj Close']

# check the length of price data history
price.describe()

,ACES,AGNG,AIQ,ANEW,AQWA,ARKF,ARKK,ARKQ,ARKW,BATT,...,VCAR,VCLO,VEGI,VFIN,WCBR,WFH,WOOD,WUGI,WWOW,YOLO
count,718.000000,1257.000000,749.000000,139.000000,19.000000,569.000000,1639.000000,1662.000000,1662.000000,735.000000,...,89.000000,89.000000,2330.000000,89.000000,69.000000,218.000000,3239.000000,278.000000,96.000000,517.000000
mean,40.451674,20.595142,18.832375,42.413257,15.565332,31.710579,42.039328,32.465094,47.543240,12.115934,...,12.838992,11.616221,24.863447,11.886564,22.914812,61.792922,46.837862,44.006259,26.681873,15.269677
std,20.068181,3.843397,4.948656,2.355340,0.136457,12.416958,30.099693,17.229812,36.912885,2.815209,...,1.397953,0.970815,3.846773,1.638937,1.562174,8.066844,15.170888,8.821878,1.179040,5.564856
min,20.787355,14.271149,12.044546,36.954193,15.296000,17.693396,13.863289,13.743156,13.829743,5.966866,...,10.789000,9.910000,18.202040,9.352200,20.549999,49.829693,14.486030,24.396999,24.474001,5.790076
25%,26.364106,18.198317,15.343655,40.607327,15.509500,21.721891,19.491617,19.112825,19.806904,10.344819,...,11.726000,10.746000,22.337907,10.462000,21.729000,53.535914,35.369858,37.734249,25.655250,11.039744
50%,30.957672,20.291855,16.550764,42.529999,15.576000,24.956055,36.920673,30.766008,41.866030,11.206928,...,12.266000,11.697000,24.073727,11.203000,22.548000,60.925390,44.721169,45.126501,26.620999,12.638284
75%,49.846800,23.021683,22.404268,44.357500,15.652500,41.453953,45.993221,34.695930,51.798170,14.091374,...,14.120000,12.529000,26.917101,13.225000,23.830000,69.547503,57.360104,50.699000,27.270750,20.821682
max,100.257912,29.490000,30.620001,47.064999,15.770000,63.599998,156.580002,99.199997,187.470001,19.487913,...,15.635000,13.105000,42.300201,15.138000,26.104000,75.919998,95.629997,61.134998,29.555000,30.559999


In [4]:
# check the distribution by ETF price history length
count = price.describe().loc['count']
count.iplot(kind='histogram', xTitle='Days of Price History', yTitle='Number of ETFs', 
            title ='Histogram of ETFs Price History Length')

2 years of daily price looks short but could be a reasonable length for gauging the price pattern, especially given that we have gone through a market cycle since the onset of the COVID-19 crisis. I calculated the annualized returns and volatilities of ETFs with longer than 2 years of history. For ETFs with shorter than 2 years of history, I marked N/A for both annualized returns and volatilities.

In [6]:
# shortern the price data collection to 2 years of history
price2Y = yf.download(ticker, period = '2Y', progress=False)['Adj Close']

# display the first 5 rows
price2Y.head()

,ACES,AGNG,AIQ,ANEW,AQWA,ARKF,ARKK,ARKQ,ARKW,BATT,...,VCAR,VCLO,VEGI,VFIN,WCBR,WFH,WOOD,WUGI,WWOW,YOLO
Date,,,,,,,,,,,,,,,,,,,,,
2019-05-07,28.129927,20.573341,16.134563,NaN,NaN,21.872444,46.383392,34.092728,52.064159,11.196300,...,NaN,NaN,26.750879,NaN,NaN,NaN,60.105728,NaN,NaN,22.768497
2019-05-08,27.882622,20.421240,16.302631,NaN,NaN,22.011187,45.530251,33.567310,52.103642,11.115148,...,NaN,NaN,26.750879,NaN,NaN,NaN,59.506214,NaN,NaN,22.844521
2019-05-09,27.560053,20.427164,16.183996,NaN,NaN,21.865557,45.138000,33.220333,51.689026,11.013704,...,NaN,NaN,26.384823,NaN,NaN,NaN,59.448200,NaN,NaN,22.416222
2019-05-10,27.618702,20.395559,16.187952,NaN,NaN,21.923613,45.343933,33.299644,51.767998,11.090994,...,NaN,NaN,26.827942,NaN,NaN,NaN,59.641590,NaN,NaN,23.096678
2019-05-13,27.090858,19.921474,15.650131,NaN,NaN,21.077368,42.843346,31.743216,49.487602,10.748023,...,NaN,NaN,26.009134,NaN,NaN,NaN,57.417599,NaN,NaN,22.193726


In [7]:
# drop ETFs with history shortern than 2 years
price2Yclean = price2Y.dropna(axis=1)

# Calculate annualized returns and volatilities
import numpy as np
rtn = np.log(price2Yclean).diff()
rtn_ann = rtn.mean()*252
vol_ann = rtn.std()*np.sqrt(252)

# create a dataframe for annualized returns and volatilities
rtn_vol = pd.DataFrame({'Return': rtn_ann, 'Volatility':vol_ann})

# display the return / volatility dataframe
rtn_vol

,Return,Volatility
ACES,0.434750,0.400089
AGNG,0.164705,0.225248
AIQ,0.284747,0.275157
ARKF,0.401354,0.340954
ARKK,0.424166,0.429980
...,...,...
UBOT,0.329302,0.864715
URA,0.318153,0.331906
VEGI,0.229112,0.270628
WOOD,0.232191,0.327433


In [8]:
# append ETFs shorter than 2 years with N/A for returns and volatilities
ETFshort = [item for item in list(price2Y.columns) if item not in list(price2Yclean.columns)]

for i in ETFshort:
    rtn_vol=rtn_vol.append(pd.Series(name=i))
    
# display the updated ETF return / volatility dataframe that includes all 143 ETFs
rtn_vol

,Return,Volatility
ACES,0.434750,0.400089
AGNG,0.164705,0.225248
AIQ,0.284747,0.275157
ARKF,0.401354,0.340954
ARKK,0.424166,0.429980
...,...,...
VFIN,NaN,NaN
WCBR,NaN,NaN
WFH,NaN,NaN
WUGI,NaN,NaN


## Holdings data

We can now move on to collect the holding data. I noticed that thematic funds generally have strong tilts to selected sectors based on the mega trends the funds are capturing. Therefore, a good starting point could be to gather the sector allocation for each ETF. I used <a href="https://github.com/dpguthrie/yahooquery">yahooquery</a> API to extract sector allocation from Yahoo! finance for individual ETFs.

In [9]:
# import yahooquery library
from yahooquery import Ticker

# create an empty dataframe
sector =pd.DataFrame()

# collect sector allocation for each ETF via yahooquery API
for i in range(0,len(rtn_vol)):
    t = Ticker(list(rtn_vol.index)[i])
    if type(t.fund_sector_weightings) == dict:
        df = pd.Series(name=list(rtn_vol.index)[i])
    else:
        df = t.fund_sector_weightings.transpose()
    sector = sector.append(df)

# display the output
sector

,realestate,consumer_cyclical,basic_materials,consumer_defensive,technology,communication_services,financial_services,utilities,industrials,energy,healthcare
ACES,0.0401,0.0926,0.0127,0.0000,0.3196,0.0000,0.0000,0.2844,0.2261,0.0244,0.0000
AGNG,0.0520,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.9480
AIQ,0.0000,0.0885,0.0000,0.0027,0.6367,0.1782,0.0000,0.0000,0.0878,0.0000,0.0059
ARKF,0.0352,0.1446,0.0000,0.0000,0.3411,0.2393,0.2155,0.0000,0.0000,0.0000,0.0242
ARKK,0.0133,0.1167,0.0000,0.0110,0.2294,0.2945,0.0221,0.0000,0.0204,0.0000,0.2927
...,...,...,...,...,...,...,...,...,...,...,...
VFIN,0.0000,0.1509,0.0000,0.0204,0.4714,0.0815,0.2322,0.0041,0.0115,0.0000,0.0281
WCBR,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
WFH,0.0000,0.0348,0.0000,0.0000,0.8086,0.1566,0.0000,0.0000,0.0000,0.0000,0.0000
WUGI,0.0000,0.1062,0.0000,0.0000,0.6405,0.2372,0.0161,0.0000,0.0000,0.0000,0.0000


To check if the sector allocation is complete, I created a "Total Sector Allocation" column in the `sector` dataframe to calculate the sum of all sector allocations for each ETF. For those ETFs that are missing sector allocation information on Yahoo! Finance, I used <a href="https://finnhub.io/docs/api/etfs-sector-exposure">Finnhub API</a> as an alternative source to see if those missing sector allocation data are available on Finnhub. If you'd like to replicate this part of implementation, you can request a free token on Finnhub and replace the "Insert Your Token" in my code below with your token. Finnhub sets limits on API calls per minute, so if the quantity of calls is large, you may have to split the data extraction to a few batches and wait for 1 minute between each batch extraction.

In [10]:
# check if all 11 sector allocations sum up to 1
sector['Total Sector Allocation'] = sector[list(sector.columns)].sum(axis=1)

In [11]:
# select ETFs with sector allocation not adding up close to 1
sectormissing = sector.loc[abs(sector['Total Sector Allocation']-1) > 0.1]

# display those ETFs missing sector information
sectormissing

,realestate,consumer_cyclical,basic_materials,consumer_defensive,technology,communication_services,financial_services,utilities,industrials,energy,healthcare,Total Sector Allocation
ESPO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
IGF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
IRBO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
MOO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
SMOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
SOCL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
DAPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
GBLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
HDRO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IHAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


Apparently some newer ETFs do not have sector allocation data available yet on Yahoo! Finance. Finnhub has sector allocation for some of these ETFs. In the past I have noticed Finnhub sector allocation doesn't seem to have look-through capability like Yahoo! Finance does. That is, if an ETF holds other ETFs, Yahoo! Finance outputs sector allocation based on the underlying holdings of the underlying ETFs, whereas Finnhub simply categorizes the sector for these underlying ETFs as "Other. However, the ETFs I'm looking at here don't appear to have significant amount of underlying ETFs and therefore the aforementioned issue doesn't apply here. 

In [12]:
# import library
import requests

# create an empty dataframe
sector2=pd.DataFrame()

# collect sector allocation via finnhub API
for i in range(0,len(sectormissing)):
    r = requests.get('https://finnhub.io/api/v1/etf/sector?symbol='+list(sectormissing.index)[i]+'&token=c0ilfqf48v6ot9ddgke0')
    x = r.json()['sectorExposure']
    
    if len(x) ==0:
        df = pd.Series(name=list(sectormissing.index)[i])
    else:
        df = pd.DataFrame(x).transpose()
        df.columns=df.iloc[1]
        df = df.drop(df.index[1])
        df.index=[list(sectormissing.index)[i]]
    
    sector2 = sector2.append(df)

# display sector2
sector2

,Communication Services,Information Technology,Consumer Discretionary,Industrials,Utilities,Energy,Health Care,Consumer Staples,Materials,Other,Real Estate,Financials
ESPO,71.19,23.42,4.38,1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IGF,0.22,NaN,NaN,36.34,36.86,20.83,NaN,NaN,NaN,NaN,NaN,NaN
IRBO,19.93,55.34,9.87,10.46,NaN,0.19,1.62,NaN,NaN,NaN,NaN,NaN
MOO,NaN,NaN,3.65,20.48,NaN,NaN,24.53,27.37,21.73,0.05,NaN,NaN
SMOG,NaN,36.39,16.07,29.12,9.38,NaN,NaN,NaN,3.95,NaN,0.35,NaN
SOCL,99.51,NaN,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DAPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HDRO,NaN,NaN,NaN,83.67,NaN,3.89,NaN,3.54,8.52,NaN,NaN,NaN
IHAK,NaN,88.59,NaN,11.1,NaN,NaN,NaN,NaN,NaN,0.11,NaN,NaN


In [13]:
# create a Total Sector Allocation column to sum up sector allocations
sector2['Total Sector Allocation'] = sector2[list(sector2.columns)].sum(axis=1)

# display sector2
sector2

,Communication Services,Information Technology,Consumer Discretionary,Industrials,Utilities,Energy,Health Care,Consumer Staples,Materials,Other,Real Estate,Financials,Total Sector Allocation
ESPO,71.19,23.42,4.38,1.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.01
IGF,0.22,NaN,NaN,36.34,36.86,20.83,NaN,NaN,NaN,NaN,NaN,NaN,94.25
IRBO,19.93,55.34,9.87,10.46,NaN,0.19,1.62,NaN,NaN,NaN,NaN,NaN,97.41
MOO,NaN,NaN,3.65,20.48,NaN,NaN,24.53,27.37,21.73,0.05,NaN,NaN,97.81
SMOG,NaN,36.39,16.07,29.12,9.38,NaN,NaN,NaN,3.95,NaN,0.35,NaN,95.26
SOCL,99.51,NaN,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99.78
DAPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
GBLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
HDRO,NaN,NaN,NaN,83.67,NaN,3.89,NaN,3.54,8.52,NaN,NaN,NaN,99.62
IHAK,NaN,88.59,NaN,11.1,NaN,NaN,NaN,NaN,NaN,0.11,NaN,NaN,99.80


Finnhub has sector data available for four of the ETFs.

I like the 11-GICS-Sector naming convention that Finnhub uses, and therefore, convert the Yahoo! Finance sector naming convention to match that of Finnhub. I then combine the Yahoo! Finance sector data, Finnhub sector data, and the returns/volatilities data to create the final dataset named `dataset`.

In [14]:
# map the Yahoo! Finance sector naming convention to Finnhub sector naming convention 
sector_yahoo = sector.rename(columns={'communication_services':'Communication Services',
                                       'consumer_cyclical':'Consumer Discretionary',
                                       'industrials':'Industrials',
                                       'basic_materials':'Materials',
                                       'energy':'Energy',
                                       'consumer_defensive':'Consumer Staples',
                                       'technology': 'Information Technology',
                                       'healthcare': 'Health Care', 
                                       'financial_services':'Financials', 
                                       'realestate':'Real Estate', 
                                       'utilities':'Utilities'})

In [15]:
# convert sector 2 to the same unit of sector_yahoo and drop the "Other" column
sector_finnhub = sector2.drop('Other',axis=1)/100

In [16]:
# update the sector information from Finnhub API to the sector_yahoo dataframe
sector_yahoo.update(sector_finnhub)

# change the values in the dataframe to float
sector_yahoo=sector_yahoo.astype(float) 

# mark the "Total Sector Allocation" to NaN if 0
sector_yahoo['Total Sector Allocation'] = sector_yahoo['Total Sector Allocation'].replace(0,np.nan) 

In [17]:
# concatenate the return/volatility data with the sector data
ETFnew = pd.concat([ETF, rtn_vol,sector_yahoo],axis=1)

# reindex the dataframe with "Fund Name"
dataset = ETFnew.set_index('Fund Name', drop=True)

# Exploratory Data Analysis

The complete dataset has these 19 variables:

In [18]:
# check the datatypes of each variable
dataset.dtypes

Issuer                      object
AUM                         object
Expense Ratio              float64
3-Mo TR                     object
Segment                     object
Return                     float64
Volatility                 float64
Real Estate                float64
Consumer Discretionary     float64
Materials                  float64
Consumer Staples           float64
Information Technology     float64
Communication Services     float64
Financials                 float64
Utilities                  float64
Industrials                float64
Energy                     float64
Health Care                float64
Total Sector Allocation    float64
dtype: object

Here are the definitions of each variable in the dataset:
* Issuer: the issuer of the ETF
* AUM: assets under management in USD
* Expense Ratio: cost of investing in the ETF
* 3-Mo TR: 3-month trailing return
* Segment: categorization by etf.com
* Return: annualized return for the past 2 years. For ETFs with less than 2 years of history, the return is N/A.
* Volatility: annualized volatility for the past 2 years. For ETFs with less than 2 years of history, the volatility is N/A.
* Real Estate, Consumer Discretionary, Materials, Consumer Staples, Information Technology, Communication Services, Financials, Utilities, Industrials, Energy, Health Care: allocations to respective sectors
* Total Sector Allocation: sum of individual sector allocations

Of these 143 ETFs, 54 are missing price data (Return = 54) and 5 are missing sector data (Total Sector Allocation = 5). ETFs missing either price data or sector data will be removed.

In [19]:
# check for missing values
dataset.isnull().sum()

Issuer                      0
AUM                         0
Expense Ratio               0
3-Mo TR                     0
Segment                     0
Return                     54
Volatility                 54
Real Estate                 8
Consumer Discretionary      4
Materials                   7
Consumer Staples            8
Information Technology      5
Communication Services      5
Financials                  9
Utilities                   7
Industrials                 3
Energy                      7
Health Care                 7
Total Sector Allocation     2
dtype: int64

Issuer, AUM, Expense Ratio, 3-Mo TR, and Segment are not relevant and will be removed. Note that etf.com has its own categorization on the thematic ETFs via the "Segment" variable. In total, there are 46 segments. This variable could be relevant if Natural Language Processing is applied, but for now I will just focus on the price and holdings data.

In [20]:
# check number of segments
segments = dataset.groupby('Segment').size()

# plot the issuers
segments.iplot(kind='bar', title='ETF Segments', xTitle='Number of ETFs', orientation='h')

As suspected, some sectors (Technology, Industrials, and Consumer Discretionary) are more popular than the others in the thematic investing world.

In [21]:
# create a sector list
sector_list = ['Real Estate', 'Consumer Discretionary', 'Materials', 'Consumer Staples', 'Information Technology',
               'Communication Services','Financials','Utilities','Industrials', 'Energy', 'Health Care']

# plot the histogram of sector allocations
dataset[sector_list].iplot(kind='histogram', title ='Histogram of Sector allocation',subplots=True)

# Data wrangling

After removing irrelevant variables and missing data points, we are left with 89 ETFs and 13 variables. The following table shows selected variables and we can see they have different scales.

In [22]:
# remove Issuer, AUM, Expense Ratio, 3-Mo TR and Segment Column
dataset2 = dataset.drop(['Issuer', 'AUM', 'Expense Ratio', '3-Mo TR', 'Segment'], axis=1)

In [23]:
# remove ETFs missing price data
dataset3 = dataset2[dataset2['Return'].notna()]

# remove ETFs missing sector data
dataset4 = dataset3[dataset3['Total Sector Allocation'].notna()]

# sectors with N/A should actually be 0
dataset4[sector_list]=dataset4[sector_list].fillna(0)

# drop the Total Sector Allocation column
dataset_final = dataset4.drop(['Total Sector Allocation'],axis=1)

In [24]:
# display the first five rows of the dataset
dataset_final.head()

,Return,Volatility,Real Estate,Consumer Discretionary,Materials,Consumer Staples,Information Technology,Communication Services,Financials,Utilities,Industrials,Energy,Health Care
Fund Name,,,,,,,,,,,,,
ARK Innovation ETF,0.424166,0.429980,0.0133,0.1167,0.0000,0.0110,0.2294,0.2945,0.0221,0.0000,0.0204,0.0000,0.2927
ARK Next Generation Internet ETF,0.476103,0.393839,0.0326,0.1880,0.0000,0.0094,0.3208,0.3168,0.0748,0.0000,0.0000,0.0000,0.0576
iShares Global Clean Energy ETF,0.390705,0.377971,0.0000,0.0000,0.0106,0.0000,0.1966,0.0000,0.0000,0.5608,0.2250,0.0070,0.0000
FlexShares Morningstar Global Upstream Natural Resources Ind…,0.141864,0.295510,0.0135,0.0000,0.5034,0.1543,0.0000,0.0000,0.0005,0.0457,0.0085,0.2741,0.0000
ARK Fintech Innovation ETF,0.401354,0.340954,0.0352,0.1446,0.0000,0.0000,0.3411,0.2393,0.2155,0.0000,0.0000,0.0000,0.0242


In [25]:
# describe the data
dataset_final.describe()

,Return,Volatility,Real Estate,Consumer Discretionary,Materials,Consumer Staples,Information Technology,Communication Services,Financials,Utilities,Industrials,Energy,Health Care
count,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000,89.000000
mean,0.245428,0.318821,0.024197,0.105833,0.094169,0.041090,0.224549,0.074581,0.036721,0.100458,0.186128,0.052563,0.057957
std,0.136423,0.086153,0.079511,0.154984,0.164688,0.095168,0.256215,0.169129,0.092800,0.189447,0.197897,0.130109,0.148436
min,-0.219544,0.182013,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.172213,0.273782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010200,0.000000,0.000000
50%,0.229112,0.296295,0.000000,0.036500,0.010500,0.000000,0.158600,0.000000,0.000000,0.000000,0.131900,0.000000,0.000000
75%,0.336411,0.340954,0.008100,0.134000,0.088600,0.029700,0.363900,0.071100,0.003400,0.093800,0.322200,0.023500,0.042200
max,0.537810,0.864715,0.643000,0.714100,0.591500,0.571900,0.929500,0.995100,0.432900,0.924800,0.745600,0.700800,0.948000


# Data transformation

K Means Clustering aims to group data points (ETFs) into clusters. Each of these clusters will have a center, and each of the data points is associated with the center that is closest to it. Mathematically, we want to minimize the distance of individual data point to its closet center. Using Euclidean distance, the distance can be calculated as:

$$\mbox{Distance}^{(n,k)} = \sqrt{\sum_{m=1}^{M}(x^n_m-c^k_m)^2}$$

, where:


* $x^n = n$th data point for $n = 1$ to $N$
* $c^k = k$th center for $k = 1$ to $K$
* $m = m$th feature for $m = 1$ to $M$

Each data point $n$ is then associated with the nearest center:

$$\underset{k}{\mbox{argmin}}\mbox{Distance}^{(n, k)}$$

Given that K Means Clustering uses distance to identify similar points for clustering, it does not work well when the scales across features are widely different, or with higher dimension data like the 13 variables we have here. Therefore, feature scaling and dimension reduction should be explored.

## Feature scaling

The idea of feature scaling is to make sure that features are on a similar scale. Here, I use StandardScaler, which is calculated as:

$$x_\mbox{i_scaled} =\frac{x_i-\bar{x}}{\sigma}$$

, where:
* $x_i$ is the $i$th data point of the feature
* $\bar{x}$ is the mean
* $\sigma$ is the standard deviation.

It rescales the features to be approximately standard normally distributed. The data is transformed to have a mean of 0 and a standard deviation of 1.
After rescaling, we can see the mean and standard deviation of each feature are close to 0 and 1, respectively.

In [26]:
# scale the dataset
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dataset_final)
dataset_scaled = scaler.transform(dataset_final)

In [27]:
# describe the scaled dataset
data=pd.DataFrame(dataset_scaled, columns = dataset_final.columns, index=dataset_final.index)
data.describe()

,Return,Volatility,Real Estate,Consumer Discretionary,Materials,Consumer Staples,Information Technology,Communication Services,Financials,Utilities,Industrials,Energy,Health Care
count,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01,8.900000e+01
mean,2.869116e-17,3.156027e-16,-1.871162e-17,-9.979533e-17,9.979533e-18,-8.607347e-17,-5.987720e-17,-2.681999e-17,2.494883e-17,-6.237208e-17,-2.058279e-16,6.860929e-18,5.613487e-18
std,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00,1.005666e+00
min,-3.427618e+00,-1.596960e+00,-3.060410e-01,-6.867296e-01,-5.750386e-01,-4.342066e-01,-8.813756e-01,-4.434702e-01,-3.979465e-01,-5.332776e-01,-9.458570e-01,-4.062806e-01,-3.926661e-01
25%,-5.397113e-01,-5.257396e-01,-3.060410e-01,-6.867296e-01,-5.750386e-01,-4.342066e-01,-8.813756e-01,-4.434702e-01,-3.979465e-01,-5.332776e-01,-8.940231e-01,-4.062806e-01,-3.926661e-01
50%,-1.202709e-01,-2.629406e-01,-3.060410e-01,-4.498874e-01,-5.109205e-01,-4.342066e-01,-2.588571e-01,-4.434702e-01,-3.979465e-01,-5.332776e-01,-2.755738e-01,-4.062806e-01,-3.926661e-01
75%,6.706991e-01,2.583630e-01,-2.035915e-01,1.827736e-01,-3.400423e-02,-1.203597e-01,5.469628e-01,-2.069797e-02,-3.611009e-01,-3.534587e-02,6.914839e-01,-2.246394e-01,-1.067572e-01
max,2.155348e+00,6.372238e+00,7.826677e+00,3.946944e+00,3.036946e+00,5.609197e+00,2.766991e+00,5.473557e+00,4.293358e+00,4.375968e+00,2.843098e+00,5.010493e+00,6.030121e+00


## Dimension reduction

Principal Component Analysis (PCA) is a dimensionality-reduction method that transforms a large set of variables into a smaller one that still contains most of the information in the large set. Mathematically, for a dataset $X$ with $M$ features, we want to find a sequence of $P$ orthogonal unit vectors $\alpha_p$, for $p = 1$ to $P$, that can explain as much as possible of the variance in the data:

1. Taking linear combinations of the M original variables: $X\alpha_p$, for $p = 1$ to $P$, where $\alpha_p$ is an $M\times 1$ column vector 
2. Each of these linear combinations explains as much as possible of the variance in the data: $\underset{\alpha_p}{\mbox{max}}V_p$, where $V_p = \mathbb{V}(X\alpha_p)$, with $V_i \geq V_j$ if $i<j$.
3. Each linear combination has unit length: $\alpha_p^{\mbox{T}}\alpha_p = 1$ $\forall p$
4. Each linear combination is uncorrelated with all the others: $\alpha_i^{\mbox{T}}\alpha_j = 0$ $\forall i\neq j$.


As an example, assuming we want to conduct a PCA with 2 components, i.e. $P = 2$. We start by focusing on the first linear combination with maximum variance. This is a constrained optimization problem where our goal is:

$$\underset{\alpha_1}{\mbox{max}}\mathbb{V}(X\alpha_1)  \qquad \mbox{  s.t.  }\qquad \alpha_1^{\mbox{T}}\alpha_1 = 1$$

<center>, where $\alpha_1$ is an $M\times 1$ column vector</center>


The Lagrangian is:

$$ L = (X\alpha_1)^{\mbox{T}}X\alpha_1 - \lambda_1(\alpha_1^{\mbox{T}}\alpha_1-1)$$
$$ = \alpha_1^{\mbox{T}}\Sigma\alpha_1 - \lambda_1(\alpha_1^{\mbox{T}}\alpha_1-1)$$

<center>, where $\Sigma$ is the variance matrix </center> 

Take the first derivative and set to zero:

$$\frac{\partial L}{\partial \alpha_1} = \Sigma\alpha_1 - \lambda_1\alpha_1 = 0 \rightarrow (\Sigma - \lambda_1I_M)\alpha_1 = 0$$

<center>, where $I_M$ is an identity matrix of size $M$ </center>

Recall our goal is to maximize $\mathbb{V}(X\alpha_1)$. This is equal to maximizing $\lambda_1$ because:

$$\mathbb{V}(X\alpha_1)=\alpha_1^{\mbox{T}}\Sigma\alpha_1 = \lambda_1\alpha_1^{\mbox{T}}\alpha_1 = \lambda_1$$

Therefore, $\lambda_1$ = largest eigenvalue, $\alpha_1$ = corresponding eigenvector.  

To find the second linear combination, we need to take a further constraint into account:

$$\underset{\alpha_2}{\mbox{max}}\mathbb{V}(X\alpha_2)  \qquad \mbox{  s.t.  }\qquad \alpha_2^{\mbox{T}}\alpha_2 = 1 \mbox{ and } \alpha_2^{\mbox{T}}\alpha_1=0$$

The Lagrangian is:

$$ L = (X\alpha_2)^{\mbox{T}}X\alpha_2 - \lambda_2(\alpha_2^{\mbox{T}}\alpha_2-1)-\theta\alpha_2^{\mbox{T}}\alpha_1$$
$$ = \alpha_2^{\mbox{T}}\Sigma\alpha_2 - \lambda_2(\alpha_2^{\mbox{T}}\alpha_2-1)-\theta\alpha_2^{\mbox{T}}\alpha_1$$

Take the first derivative and set to zero:

$$\frac{\partial L}{\partial \alpha_2} = \Sigma\alpha_2 - \lambda_2\alpha_2 - \theta_2\alpha_1 = 0$$ 
$$\rightarrow\alpha_1^{\mbox{T}}\Sigma\alpha_2 - \lambda_2\alpha_1^{\mbox{T}}\alpha_2 - \theta_2\alpha_1^{\mbox{T}}\alpha_1 = 0$$
$$\rightarrow 0 + 0 + \theta_2\times 1 = 0$$
$$\rightarrow \theta = 0$$

Therefore,
$$(\Sigma - \lambda_2I_M)\alpha_2 = 0$$

$\lambda_2$ = second largest eigenvalue, $\alpha_2$ = corresponding eigenvector.  


From the PCA performed on the thematic ETF dataset below, we can see as the number of components goes up, the better the variance can be explained. 

In [28]:
# import libraries
from sklearn.decomposition import PCA

# initiate an empty dataframe
pca_var = pd.DataFrame()

# performance PCA for a varying number of components
for i in range(1,len(dataset_final.columns)+1):
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(dataset_scaled)
    var = pd.Series(np.sum(pca.explained_variance_ratio_), name=i)
    pca_var=pca_var.append(var)

# plot   
pca_var.iplot(title='Principal Compoent Analysis on Thematic ETF dataset', xTitle='Number of Components', yTitle='Cumulative Explained Variance')

Usually we would set a minimum threshold for variance explained and then select the smallest number of components that can meet the threshold. However, in this project, I will explore the impact that the number of components can have on K Means Clustering. 

# K Means Clustering

In K Means Clustering, adding up all the squared distances to the nearest center gives us a measure of total distance called "error". If we plot this error against the number of clusters K, we get a scree plot. We can see that there's no "elbow" in the scree plots beyond 4 PCA components. An "elbow" is where the error falls dramatically and then levels off. The x-axis of the elbow is the best number of clusters we are looking for. If the error only gradually falls (no elbow) then there is no obvious best K using this methodology.

In [29]:
# import libraries
from sklearn.cluster import KMeans
from plotly.subplots import make_subplots
import math

# create subplots
fig=make_subplots(rows=math.ceil(len(dataset_final.columns)/3), cols=3)

# perform PCA for a varying number of components
for i in range(1,len(dataset_final.columns)+1):
    pca = PCA(n_components=i)
    pca_result = pca.fit_transform(dataset_scaled)
    
    # perform K Means Clustering by varying number of clusters
    error, cluster =[], []
    for k in range(1,20):
        km = KMeans(n_clusters=k)
        km = km.fit(pca_result)
        error.append(km.inertia_)
        cluster.append(k)
    
    # plotting
    fig.add_scatter(y=error, row=int(i/3)+1, col=i%3+1, name=str(i)+' PCA components')

# output plots
fig.update_layout(title='K Means Clustering Scree Plots by varying number of PCA components')
fig.show()




The higher the number of PCA components, the more information I get to preserve from the original features. Therefore, I am incentivized to maximize the number of PCA components among scree plots with an elbow. In this case, 4 PCA components and 4 clusters looks like the optimal option. I cannot plot a 6-dimension graph, so I randomly use two of the features for visualization. Each color represents one cluster. 

In [31]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# PCA analysis with 4 components
pca_final = PCA(n_components=4)
pca_final_result = pca_final.fit_transform(dataset_scaled)

# K Means Clustering with 4 clusters
km_final = KMeans(n_clusters=4).fit(pca_final_result)

# plotting
dataset_final['label'] = km_final.labels_
dataset_final['ticker'] = dataset_final.index
dataset_final[['label']] = dataset_final[['label']].astype('float64', copy=False) #converting label column to float64 to be able to use in plotly
dataset_final.iplot(kind='scatter', x='Volatility',y='Return', mode ='markers', categories='label', text='ticker',
                   title='Thematic ETF Clusters', xTitle='Volatility', yTitle='Return', legend=False)


Although this 2-D graph cannot fully illustrate the similarities, we can see that those ETFs seem to cluster according to Return and Volatility to some extent. The table below more clearly lists out the ETFs in each of the clusters.

In [32]:
# create an empty dataframe
labels=pd.DataFrame()

# generate a table for ETFs within each cluster
for i in range(0,max(km_final.labels_)+1):
    label = pd.Series(dataset_final.loc[dataset_final['label'] == i].index, name='Cluster '+str(i+1))
    labels=pd.concat([labels,label], axis=1)

# replace NaN with blank cell
labels = labels.replace(np.nan,'',regex=True)

# print the table
labels

,Cluster 1,Cluster 2,Cluster 3,Cluster 4
0,ETFMG Alternative Harvest ETF,ARK Innovation ETF,iShares Global Clean Energy ETF,FlexShares Morningstar Global Upstream Natural...
1,AdvisorShares Pure Cannabis ETF,ARK Next Generation Internet ETF,iShares Global Infrastructure ETF,SPDR S&P Global Natural Resources ETF
2,ProShares Pet Care ETF,ARK Fintech Innovation ETF,Global X U.S. Infrastructure Development ETF,VanEck Vectors Agribusiness ETF
3,Global X Aging Population ETF,First Trust NASDAQ Cybersecurity ETF,FlexShares STOXX Global Broad Infrastructure I...,Global X Uranium ETF
4,The Long-Term Care ETF,ARK Autonomous Technology & Robotics ETF,Invesco Water Resources ETF,SPDR S&P North American Natural Resources ETF
5,,Invesco Solar ETF,First Trust Water ETF,iShares Global Timber & Forestry ETF
6,,Global X Lithium & Battery Tech ETF,Invesco S&P Global Water Index ETF,iShares North American Natural Resources ETF
7,,First Trust NASDAQ Clean Edge Green Energy Ind...,iShares U.S. Infrastructure ETF,Columbia Emerging Markets Consumer ETF
8,,Global X Robotics & Artificial Intelligence ETF,SPDR S&P Global Infrastructure ETF,Amplify Lithium & Battery Technology ETF
9,,Invesco WilderHill Clean Energy ETF,First Trust Global Wind Energy ETF,Invesco MSCI Global Timber ETF


In [33]:
# export the tables to png file
import dataframe_image as dfi
dfi.export(labels,'labels.png')

A browse of individual ETFs reveals the mega trends within each of the clusters: 
* Cluster 1: natural resources and agriculture
* Cluster 2: energy transition and automation
* Cluster 3: clean energy and infrastructure
* Cluster 4: aging population

Cluster 2 is a big group. Another K Means Clustering may be performed to this cluster if we want to further break it down to smaller clusters.

# Closing Thoughts

By simply supplying price and sector data of thematic ETFs, K Means Clustering appears to effectively identify 4 distinctive clusters. The popular mega trends are:  natural resources and agriculture, energy transition and automation, clean energy and infrastructure, and aging population. The "energy transition and automation" cluster is significantly larger than the other 3 clusters and another K Means Clustering can be conducted if a further split is needed. The research can potentially be improved by incorporating more thematic funds and individual stock holdings. Applying Natural Language Processing on the names of the ETFs may also shed some insight.